<img src="https://cognitiveclass.ai/wp-content/uploads/2018/08/Data-Science-Professional-Certificate-300x300.png">

<h1 style="text-align:center">Business Problem</h1>

<p>The main objective of this project is to cluster the neighbourhoods of a city called 'Bangalore', which has various forms of lifes around various corners of the city. Here I made a cluster analysis for the major towns to see which are similar and which are not</p>
<p>The second objective is to cluster these cities where a person can succesfully open a restuarant. So as the data Available I have considered only 2 factors. 
<ul>
    <li> Competition - So area with less restaurants are preferred</li>
    <li> Markets - Availablity of required products, as the markets nearby</li>
</ul></p>

In [615]:
import pandas as pd
import html5lib
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim
import folium
import requests 
from pandas.io.json import json_normalize
import json
from math import radians, sin, cos, acos
import matplotlib.cm as cm
import matplotlib.colors as colors

<h1 style="text-align:center">Data Collection</h1>

Inorder to get major neighbourhoods in bangalore, I searched for dataset which has complete zip code and corresponding location details

In [616]:
tables= pd.read_html("https://www.mapsofindia.com/pincode/india/karnataka/bangalore/")
table=tables[0]

In [617]:
table.head()

,Pincode Details,Pincode Details.1,Pincode Details.2,Pincode Details.3
0,Location,Pincode,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore


In [618]:
# Removing first row and putting it as the header
nh=table.iloc[0]
table=table[1:]
table.columns=nh
table.head()

,Location,Pincode,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore
5,Air Force hospital,560007,Karnataka,Bangalore


In [619]:
table.shape

(294, 4)

In [620]:
table['lat']=''
table['long']=''
table.head()

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Location,Pincode,State,District,lat,long
1,A F station yelahanka,560063,Karnataka,Bangalore,,
2,Adugodi,560030,Karnataka,Bangalore,,
3,Agara,560034,Karnataka,Bangalore,,
4,Agram,560007,Karnataka,Bangalore,,
5,Air Force hospital,560007,Karnataka,Bangalore,,


###### Inorder to get the latitude and longitude values, I have to Iteratively request for geo location using geopy package (as the dataset wasn't that easy to get). Sometimes the iteration will break dueto server response timeout. so I made a temporary dataframe 'rem' which will store all row which is not processed during the request. A try, exception has given in the iteration in case the server timeout occures it will simply ignore that row and proceed with next iteration cycle. At the end of the iteration, We check for any remaining rows in 'rem' dataframe, and repeate process untill rem table is empty.

In [629]:
rem=table[table.lat=='']
rem.head()

,Location,Pincode,State,District,lat,long


In [628]:
count=rem.shape[0]
for loc in rem['Location']:
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(loc)
    except :
        continue
    if(location):
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude = 'null'
        longitude = 'null'
    count=count-1;
    table.loc[table['Location'] == loc, 'lat'] = latitude
    table.loc[table['Location'] == loc, 'long'] = longitude
    print(count)
    

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


5
4
3
2
1
0


In [630]:
rem.shape

(0, 6)

In [631]:
table

,Location,Pincode,State,District,lat,long
1,A F station yelahanka,560063,Karnataka,Bangalore,null,null
2,Adugodi,560030,Karnataka,Bangalore,12.9428,77.6104
3,Agara,560034,Karnataka,Bangalore,27.6706,85.0162
4,Agram,560007,Karnataka,Bangalore,45.8132,15.977
5,Air Force hospital,560007,Karnataka,Bangalore,55.245,-162.759
6,Amruthahalli,560092,Karnataka,Bangalore,null,null
7,Anandnagar,560024,Karnataka,Bangalore,18.7111,73.3017
8,Anekal,562106,Karnataka,Bangalore,12.7086,77.6994
9,Anekalbazar,562106,Karnataka,Bangalore,null,null
10,Arabic College,560045,Karnataka,Bangalore,8.88847,-12.0308


###### The 'null' rows can be removed as we have no use of'em

In [632]:
# Null columns are removed
df=table[table.lat!='null']
print(df.shape[0])
df.head()

211


,Location,Pincode,State,District,lat,long
2,Adugodi,560030,Karnataka,Bangalore,12.9428,77.6104
3,Agara,560034,Karnataka,Bangalore,27.6706,85.0162
4,Agram,560007,Karnataka,Bangalore,45.8132,15.977
5,Air Force hospital,560007,Karnataka,Bangalore,55.245,-162.759
7,Anandnagar,560024,Karnataka,Bangalore,18.7111,73.3017


###### Some Locations like 'Air Force hospital' is referring to location in other countries. and Places like 'Mahatma Gandi Road' which are also there in other parts of India. So we can remove location which have distance more than 25km from bangalore city. 

In [282]:
# some fileds contains lat and long of other places, from other countries. which is considered as noise.
# Removing cells with KM > 25 by checking each cells
count=0
for loc,lat,lon in zip(df['Location'],df['lat'],df['long']):
    slat = radians(float(12.9791198))
    slon = radians(float(77.5912997))
    elat = radians(float(lat))
    elon = radians(float(lon))
    dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    if(dist>25):
        df.loc[df['Location'] == loc, 'lat'] = 'null'
        df.loc[df['Location'] == loc, 'long'] = 'null'
        count+=1
        print('{} \t {}'.format(loc,dist))
        
print('total = {}'.format(count))

Anekal 	 32.2788497723
Attibele 	 29.6559507771
Attur 	 188.292599346
Bagalur 	 34.0082967129
Bannerghatta 	 49.8573847257
Bhattarahalli 	 139.401809798
Bidaraguppe 	 28.5833959076
Bommanahalli 	 33.8423726302
Chikkanahalli 	 42.8810819038
Dasarahalli 	 29.7048858066
Devanagundi 	 26.8901948483
Devasandra 	 57.4119295972
Dommasandra 	 94.1258766737
Handenahalli 	 25.5744319093
Hulimavu 	 130.273440744
Kamagondanahalli 	 124.592408931
Kannamangala 	 53.7407815313
Kenchanahalli 	 92.0544519813
Koramangala 	 38.9360680849
Kothanur 	 52.4257508907
Krishnarajapuram 	 92.9053100361
Krishnarajapuram R s 	 92.9053100361
Madivala 	 34.9885550975
Mahadevapura 	 198.555582737
Mayasandra 	 37.494201949
Medihalli 	 30.8610277705
Medimallasandra 	 26.0485340482
Nagasandra 	 74.434699802
Nagavara 	 55.8535416213
Tavarekere 	 69.6019094019
Vanakanahalli 	 35.9217224877
Vidyanagara 	 160.671859642
Yadavanahalli 	 82.3224248928
total = 33


33 rows are removed

In [633]:
df=df[df.lat!='null']
print(df.shape[0])
df.head()

211


,Location,Pincode,State,District,lat,long
2,Adugodi,560030,Karnataka,Bangalore,12.9428,77.6104
3,Agara,560034,Karnataka,Bangalore,27.6706,85.0162
4,Agram,560007,Karnataka,Bangalore,45.8132,15.977
5,Air Force hospital,560007,Karnataka,Bangalore,55.245,-162.759
7,Anandnagar,560024,Karnataka,Bangalore,18.7111,73.3017


In [634]:
df=df.reset_index(drop=True)
df.head()

,Location,Pincode,State,District,lat,long
0,Adugodi,560030,Karnataka,Bangalore,12.9428,77.6104
1,Agara,560034,Karnataka,Bangalore,27.6706,85.0162
2,Agram,560007,Karnataka,Bangalore,45.8132,15.977
3,Air Force hospital,560007,Karnataka,Bangalore,55.245,-162.759
4,Anandnagar,560024,Karnataka,Bangalore,18.7111,73.3017


###### We can drop columns State and District as they won't change for the entire dataset

In [287]:
df=df.drop(['State','District'],axis=1)

In [288]:
df.head()

,Location,Pincode,lat,long
0,Aranya Bhavan,560003,13.0106,77.5708
1,Austin Town,560047,12.9613,77.6153
2,Avalahalli,560026,13.0353,77.7366
3,Bagalgunte,560073,13.0566,77.5048
4,Banashankari Iii stage,560085,12.9327,77.5463


###### As the entire process of collecting geolocation and processing it bit time consuming. I have saved this data into a csv for quick retrievals

In [289]:
#save collected data to csv
df.to_csv('Bangalore.csv', sep=',', header=True, index=False)

In [290]:
df2=pd.read_csv('Bangalore.csv')
df2

,Location,Pincode,lat,long
0,Aranya Bhavan,560003,13.010615,77.570820
1,Austin Town,560047,12.961274,77.615294
2,Avalahalli,560026,13.035278,77.736565
3,Bagalgunte,560073,13.056649,77.504822
4,Banashankari Iii stage,560085,12.932708,77.546254
5,Banaswadi,560043,13.014162,77.651854
6,Bangalore Bazaar,560001,13.012267,77.553462
7,Bangalore City,560002,12.979120,77.591300
8,Bangalore Corporation building,560002,12.974101,77.627469
9,Bangalore Fort,560002,12.962941,77.575760


###### Getting the coordinates for Bangalore city and showing the points we collected in the map

In [292]:
address = 'Bangalore'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


In [337]:
bang_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df2['lat'], df2['long'], df2['Location']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bang_map)  
    
bang_map

###### Using foursqure API for collecting neighbourhood informations

In [635]:
CLIENT_ID = 'WCBNIU4Q5D3ZFWMLJO3EAYHIAKD2IKEJ0ZJDOJTWRN4EKOMW' # your Foursquare ID
CLIENT_SECRET = 'A2NPSQAHQIWHTVV4ZLVDSDTXEBIVCZIN551WXPHEEZLF15IB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WCBNIU4Q5D3ZFWMLJO3EAYHIAKD2IKEJ0ZJDOJTWRN4EKOMW
CLIENT_SECRET:A2NPSQAHQIWHTVV4ZLVDSDTXEBIVCZIN551WXPHEEZLF15IB


In [636]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbuorhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [637]:
bang_venues = getNearbyVenues(names=df2['Location'],
                                   latitudes=df2['lat'],
                                   longitudes=df2['long']
                                  )

Aranya Bhavan
Austin Town
Avalahalli
Bagalgunte
Banashankari Iii stage
Banaswadi
Bangalore Bazaar
Bangalore City
Bangalore Corporation building
Bangalore Fort
Bangalore.
Bannerghatta Road
Basaveshwaranagar
Bellandur
Benson Town
Brigade Road
Byatarayanapura
C.V.raman nagar
Chamrajpet
Chamrajpet Bazar
Chickpet
Chikkabettahalli
Chikkajala
Chikkalasandra
Chunchanakuppe
Deepanjalinagar
Dharmaram College
Doddajala
Doddakallasandra
Doddanekkundi
Domlur
Dr. ambedkar veedhi
G.K.v.k.
Girinagar
Goraguntepalya
Goripalya
Hsr Layout
Hennagara
Hongasandra
Hoodi
Horamavu
Hosakerehalli
Hosur Road
Hulimangala
Hunasamaranahalli
Ittamadu Layout
J P nagar
Jakkur
Jalahalli
Jalahalli East
Jalahalli Village
Jalahalli West
Jeevanahalli
Jigani
Jp Nagar iii phase
Kacharakanahalli
Kadabagere
Kadugodi
Kamakshipalya
Kathriguppe
Kendriya Sadan
Kodigehalli
Konanakunte
Koramangala I block
Kumaraswamy Layout
Kundalahalli
Madhavan Park
Mallathahalli
Maruthi Sevanagar
Mathikere
Mico Layout
Msrit
Nagarbhavi
Narayan Pillai

In [642]:
bang_venues.head(15)

,Neighbourhood,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aranya Bhavan,13.010615,77.570820,Cafe Coffee Day,13.008498,77.571723,Café
1,Aranya Bhavan,13.010615,77.570820,Ammas Pastries,13.008177,77.571189,Dessert Shop
2,Aranya Bhavan,13.010615,77.570820,chungs chinese,13.006520,77.569272,Chinese Restaurant
3,Aranya Bhavan,13.010615,77.570820,Chai Point,13.008589,77.569546,Tea Room
4,Aranya Bhavan,13.010615,77.570820,Ganesh Darshini,13.007291,77.569278,Indian Restaurant
5,Aranya Bhavan,13.010615,77.570820,Heritage Fresh Malleshwaram,13.006721,77.569081,Department Store
6,Aranya Bhavan,13.010615,77.570820,Sarkaar Restaurant,13.006733,77.568941,Indian Restaurant
7,Austin Town,12.961274,77.615294,Chaitanya Delicacy,12.963739,77.614579,Indian Restaurant
8,Avalahalli,13.035278,77.736565,hotel mandarathi,13.034399,77.734917,Breakfast Spot
9,Banaswadi,13.014162,77.651854,Tamarind,13.012676,77.647711,Indian Restaurant


In [644]:
print('There are {} uniques categories.'.format(len(bang_venues['Venue Category'].unique())))

There are 148 uniques categories.


###### The categorical Values can be converted to dummies using pandas get_dummies. Which will be useful for clustering 

In [645]:
bang_cross=pd.get_dummies(bang_venues[['Venue Category']],prefix="",prefix_sep="")
bang_cross['Neighbourhood'] = bang_venues['Neighbourhood'] 
fixed_columns = [bang_cross.columns[-1]] + list(bang_cross.columns[:-1])
bang_cross = bang_cross[fixed_columns]

In [646]:
bang_cross.shape

(770, 149)

In [647]:
bang_cross=bang_cross.groupby('Neighbourhood').mean().reset_index()
bang_cross

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Toy / Game Store,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Aranya Bhavan,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
1,Austin Town,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
2,Avalahalli,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
3,Banaswadi,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.285714,0.00,0.000000,0.0,0.000000,0.000000,0.0
4,Bangalore Bazaar,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
5,Bangalore City,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.250000,0.0,0.000000,0.000000,0.0
6,Bangalore Corporation building,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
7,Bangalore Fort,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0
8,Bangalore.,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.00,0.250000,0.0,0.000000,0.000000,0.0
9,Bannerghatta Road,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.071429,0.00,0.000000,0.0,0.000000,0.000000,0.0


In [648]:
bang_cross.shape

(89, 149)

###### Here we create a dataframe with neighbourhood name and 10 columns for the major 10 attraction of that place. This is not for clustering or something. this is to show the major 10 attractions in final result

In [465]:
def most_common(row,num):
    row=row[1:]
    row=row.sort_values(ascending=False)
    return row.index.values[0:num_top_venues]

In [466]:
num=10
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = bang_cross['Neighbourhood']

for ind in np.arange(bang_cross.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = most_common(bang_cross.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aranya Bhavan,Tea Room,Café,Indian Restaurant,Pizza Place,Chinese Restaurant,Dessert Shop,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
1,Austin Town,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
2,Avalahalli,Breakfast Spot,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop,Diner
3,Bagalgunte,Fast Food Restaurant,Pizza Place,Concert Hall,Department Store,Restaurant,Dog Run,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant
4,Banashankari Iii stage,Juice Bar,Pizza Place,Bakery,Department Store,Udupi Restaurant,Breakfast Spot,Ice Cream Shop,Coffee Shop,Convenience Store,Donut Shop
5,Banaswadi,Vegetarian / Vegan Restaurant,Convenience Store,Indian Restaurant,Bakery,BBQ Joint,Kerala Restaurant,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market
6,Bangalore Bazaar,Bowling Alley,Lounge,Multiplex,Coffee Shop,Clothing Store,Donut Shop,Italian Restaurant,Shopping Mall,Department Store,Chinese Restaurant
7,Bangalore City,Park,Hotel,Vineyard,Capitol Building,Sports Bar,Dessert Shop,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
8,Bangalore Corporation building,Bar,Light Rail Station,Yoga Studio,Flea Market,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
9,Bangalore Fort,Historic Site,Market,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop


<h1 style="text-align:center">Data Collection and Processing Phase ends here</h1>

<hr>

<h1 style="text-align:center">K-Means Algorithm</h1>
<p>Here we cluster the neighbourhoods into 5 clusters according to the facilites available</p>

In [523]:
no=5
bang_cluster=bang_cross.drop('Neighbourhood',1)
kmeans= KMeans(n_clusters=no,random_state=0).fit(bang_cluster)
kmeans.labels_[0:10]

array([0, 2, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [553]:
bang_merged=df2
bang_merged.rename(columns={"Location":"Neighbourhood"})
bang_merged.head()

,Location,Pincode,lat,long
0,Aranya Bhavan,560003,13.010615,77.570820
1,Austin Town,560047,12.961274,77.615294
2,Avalahalli,560026,13.035278,77.736565
3,Bagalgunte,560073,13.056649,77.504822
4,Banashankari Iii stage,560085,12.932708,77.546254


In [542]:
nvs=neighborhoods_venues_sorted
nvs.head()

,cluster_label,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Aranya Bhavan,Tea Room,Café,Indian Restaurant,Pizza Place,Chinese Restaurant,Dessert Shop,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
1,2,Austin Town,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
2,1,Avalahalli,Breakfast Spot,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop,Diner
3,1,Bagalgunte,Fast Food Restaurant,Pizza Place,Concert Hall,Department Store,Restaurant,Dog Run,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant
4,1,Banashankari Iii stage,Juice Bar,Pizza Place,Bakery,Department Store,Udupi Restaurant,Breakfast Spot,Ice Cream Shop,Coffee Shop,Convenience Store,Donut Shop


In [543]:
df3=df2.rename(columns={"Location":"Neighbourhood"})
df3.head()

,Neighbourhood,Pincode,lat,long
0,Aranya Bhavan,560003,13.010615,77.570820
1,Austin Town,560047,12.961274,77.615294
2,Avalahalli,560026,13.035278,77.736565
3,Bagalgunte,560073,13.056649,77.504822
4,Banashankari Iii stage,560085,12.932708,77.546254


In [544]:
df4=pd.merge(df3,nvs, on="Neighbourhood")

In [546]:
df4.head()

,Neighbourhood,Pincode,lat,long,cluster_label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aranya Bhavan,560003,13.010615,77.570820,0,Tea Room,Café,Indian Restaurant,Pizza Place,Chinese Restaurant,Dessert Shop,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop
1,Austin Town,560047,12.961274,77.615294,2,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
2,Avalahalli,560026,13.035278,77.736565,1,Breakfast Spot,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant,Donut Shop,Diner
3,Bagalgunte,560073,13.056649,77.504822,1,Fast Food Restaurant,Pizza Place,Concert Hall,Department Store,Restaurant,Dog Run,Farmers Market,Event Space,Electronics Store,Dumpling Restaurant
4,Banashankari Iii stage,560085,12.932708,77.546254,1,Juice Bar,Pizza Place,Bakery,Department Store,Udupi Restaurant,Breakfast Spot,Ice Cream Shop,Coffee Shop,Convenience Store,Donut Shop


In [554]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(no)
ys = [i + x + (i*x)**2 for i in range(no)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df4['lat'], df4['long'], df4['Neighbourhood'], df4['cluster_label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<hr>

<h1 style="text-align:center">Restuarant Location suggestion</h1>
<p> We basically go for 2 items. Markets and other restaurants. more markets means more availablity of items, which is positive. more restaurants means more options to customers, less probablity for your restaurant, obviously negative. But here we will make 2 cluster based on these factors and we check for a place which we consider favourable for our business. Then the cluster most probably going to have all the location we need</p>

###### We collect all cateogires which has restaurants in somewhere in between. and renamed all of them as 'restaurants' and kept it in one dataframe

In [650]:
rest=bang_venues[bang_venues['Venue Category'].str.contains('Restaurant')]
rest=rest[rest['Venue Category'].str.contains('Restaurant')]
rest['Venue Category']='Restaurant'
rest.head()

,Neighbourhood,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Aranya Bhavan,13.010615,77.570820,chungs chinese,13.006520,77.569272,Restaurant
4,Aranya Bhavan,13.010615,77.570820,Ganesh Darshini,13.007291,77.569278,Restaurant
6,Aranya Bhavan,13.010615,77.570820,Sarkaar Restaurant,13.006733,77.568941,Restaurant
7,Austin Town,12.961274,77.615294,Chaitanya Delicacy,12.963739,77.614579,Restaurant
9,Banaswadi,13.014162,77.651854,Tamarind,13.012676,77.647711,Restaurant


###### Making a dataframe which will have all the market locations

In [515]:
market=bang_venues[bang_venues['Venue Category']=='Market']
market

,Neighbourhood,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
66,Bangalore Fort,12.962941,77.575760,Sri Bhagyalakshmi Trading Co,12.962799,77.571726,Market
97,Basaveshwaranagar,12.986475,77.538571,Naturals,12.985436,77.542957,Market
521,Mallathahalli,12.961067,77.509909,M K Ahmed super market,12.960822,77.506931,Market
536,Mathikere,13.032888,77.557374,More Store,13.029200,77.559152,Market
564,Nagarbhavi,12.965101,77.507863,M K Ahmed super market,12.960822,77.506931,Market
568,Narayan Pillai street,12.982918,77.608474,Commercial Street,12.982042,77.607327,Market


###### Merge these dataframes to one

In [516]:
sup=pd.concat([rest,market], ignore_index=True)

In [517]:
sup

,Neighbourhood,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aranya Bhavan,13.010615,77.570820,chungs chinese,13.006520,77.569272,Restaurant
1,Aranya Bhavan,13.010615,77.570820,Sarkaar Restaurant,13.006733,77.568941,Restaurant
2,Austin Town,12.961274,77.615294,Chaitanya Delicacy,12.963739,77.614579,Restaurant
3,Bagalgunte,13.056649,77.504822,Hotel Elite Hesaraghatta,13.057818,77.506769,Restaurant
4,Bagalgunte,13.056649,77.504822,Sai Sagar Fast Food,13.056997,77.507093,Restaurant
5,Banashankari Iii stage,12.932708,77.546254,ayodhya upachara,12.928708,77.545688,Restaurant
6,Banaswadi,13.014162,77.651854,Tamarind,13.012676,77.647711,Restaurant
7,Banaswadi,13.014162,77.651854,Saffron,13.012763,77.647837,Restaurant
8,Banaswadi,13.014162,77.651854,Leela's Kitchen,13.013746,77.648765,Restaurant
9,Banaswadi,13.014162,77.651854,Idli-Yo-Idli,13.014845,77.656311,Restaurant


###### Creating dummy from categorical value. 

In [651]:
bang_rest=pd.get_dummies(sup[['Venue Category']],prefix="",prefix_sep="")
bang_rest['Neighbourhood'] = sup['Neighbourhood'] 
fixed_columns = [bang_rest.columns[-1]] + list(bang_rest.columns[:-1])
bang_rest = bang_rest[fixed_columns]

In [652]:
bang_rest.head()

,Neighbourhood,Market,Restaurant
0,Aranya Bhavan,0,1
1,Aranya Bhavan,0,1
2,Austin Town,0,1
3,Bagalgunte,0,1
4,Bagalgunte,0,1


In [654]:
bang_rest=bang_rest.groupby('Neighbourhood').mean().reset_index()
bang_rest.head(20)

,Neighbourhood,Market,Restaurant
0,Aranya Bhavan,0.000000,1.000000
1,Austin Town,0.000000,1.000000
2,Bagalgunte,0.000000,1.000000
3,Banashankari Iii stage,0.000000,1.000000
4,Banaswadi,0.000000,1.000000
5,Bangalore Bazaar,0.000000,1.000000
6,Bangalore Fort,1.000000,0.000000
7,Bannerghatta Road,0.000000,1.000000
8,Basaveshwaranagar,0.111111,0.888889
9,Brigade Road,0.000000,1.000000


###### Creating a dataframe with priority between market and restaurants

In [567]:
def most_common(row,num):
    row=row[1:]
    row=row.sort_values(ascending=False)
    return row.index.values[0:num_top_venues]

In [655]:
num=2
num_top_venues = 2

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [656]:
# create a new dataframe
rest_sorted = pd.DataFrame(columns=columns)
rest_sorted['Neighbourhood'] = bang_rest['Neighbourhood']

for ind in np.arange(bang_rest.shape[0]):
    rest_sorted.iloc[ind, 1:] = most_common(bang_rest.iloc[ind, :], num_top_venues)

rest_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue
0,Aranya Bhavan,Restaurant,Market
1,Austin Town,Restaurant,Market
2,Bagalgunte,Restaurant,Market
3,Banashankari Iii stage,Restaurant,Market
4,Banaswadi,Restaurant,Market


##### K means cluster for restaurant data

In [657]:
no2=2
bang_rest_cluster=bang_rest.drop('Neighbourhood',1)
kmeans2= KMeans(n_clusters=no2,random_state=0).fit(bang_rest_cluster)
kmeans2.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

###### merge cluster label to the dataframe

In [658]:
rest_sorted.insert(0, 'Cluster Labels', kmeans2.labels_)

###### Merge the priority dataframe and clustered dataframe togather

In [660]:
bang_rest_merged=pd.merge(sup,rest_sorted, on ="Neighbourhood")

In [661]:
bang_rest_merged

,Neighbourhood,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
0,Aranya Bhavan,13.010615,77.570820,chungs chinese,13.006520,77.569272,Restaurant,0,Restaurant,Market
1,Aranya Bhavan,13.010615,77.570820,Sarkaar Restaurant,13.006733,77.568941,Restaurant,0,Restaurant,Market
2,Austin Town,12.961274,77.615294,Chaitanya Delicacy,12.963739,77.614579,Restaurant,0,Restaurant,Market
3,Bagalgunte,13.056649,77.504822,Hotel Elite Hesaraghatta,13.057818,77.506769,Restaurant,0,Restaurant,Market
4,Bagalgunte,13.056649,77.504822,Sai Sagar Fast Food,13.056997,77.507093,Restaurant,0,Restaurant,Market
5,Banashankari Iii stage,12.932708,77.546254,ayodhya upachara,12.928708,77.545688,Restaurant,0,Restaurant,Market
6,Banaswadi,13.014162,77.651854,Tamarind,13.012676,77.647711,Restaurant,0,Restaurant,Market
7,Banaswadi,13.014162,77.651854,Saffron,13.012763,77.647837,Restaurant,0,Restaurant,Market
8,Banaswadi,13.014162,77.651854,Leela's Kitchen,13.013746,77.648765,Restaurant,0,Restaurant,Market
9,Banaswadi,13.014162,77.651854,Idli-Yo-Idli,13.014845,77.656311,Restaurant,0,Restaurant,Market


###### Display the Clustered points

In [662]:
# create map
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(no2)
ys = [i + x + (i*x)**2 for i in range(no2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bang_rest_merged['Neighbourhood Latitude'], bang_rest_merged['Neighbuorhood Longitude'], bang_rest_merged['Neighbourhood'], bang_rest_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2

###### According to manual analysis of data, Bangalore Fort is one of the best option as it has 0 for restaurants and 1 for makrets as mean. So the cluster for Bangalore Fort is 1. So the cluster contains other places which are suitable as Bangalore Fort

In [611]:
bang_rest_merged[bang_rest_merged['Cluster Labels']==1]

,Neighbourhood,Neighbourhood Latitude,Neighbuorhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
182,Mathikere,13.032888,77.557374,Food Campus,13.031899,77.559258,Restaurant,1,Restaurant,Market
183,Mathikere,13.032888,77.557374,Ammi's Biryani,13.034208,77.560674,Restaurant,1,Restaurant,Market
184,Mathikere,13.032888,77.557374,More Store,13.029200,77.559152,Market,1,Restaurant,Market
198,Nagarbhavi,12.965101,77.507863,Swathi Restaurant,12.960839,77.507716,Restaurant,1,Restaurant,Market
199,Nagarbhavi,12.965101,77.507863,M K Ahmed super market,12.960822,77.506931,Market,1,Restaurant,Market
270,Bangalore Fort,12.962941,77.575760,Sri Bhagyalakshmi Trading Co,12.962799,77.571726,Market,1,Market,Restaurant


### Finished